In [7]:
import numpy as np

X_1 = [4,3,4,4,3,3,3,3,4,4,4,5,4,3,4]
X_2 = [3028,1365,2726,2538,1318,1693,1412,1632,2875,3564,4412,4444,4278,3064,3857]
X_1 = (X_1 - np.average(X_1)) / np.std(X_1)
X_2 = (X_2 - np.average(X_2)) / np.std(X_2)
print('The rescaled feature vectors are')
print('X_1 = ', X_1)
print('X_2 = ', X_2)

The rescaled feature vectors are
X_1 =  [ 0.55901699 -1.11803399  0.55901699  0.55901699 -1.11803399 -1.11803399
 -1.11803399 -1.11803399  0.55901699  0.55901699  0.55901699  2.23606798
  0.55901699 -1.11803399  0.55901699]
X_2 =  [ 0.19474995 -1.31677383 -0.07974204 -0.25061785 -1.35949278 -1.01865008
 -1.27405488 -1.07409383  0.05568612  0.68192779  1.45268675  1.481772
  1.33089229  0.22747085  0.94823955]


Linear PCA covariance matrix

In [8]:
M=np.array([[np.dot(X_1,X_1),np.dot(X_1,X_2)],[np.dot(X_2,X_1),np.dot(X_2,X_2)]]) / (15-1)
print('The covariance matrix is', 'M = \n', M)

The covariance matrix is M = 
 [[1.07142857 0.87414684]
 [0.87414684 1.07142857]]


Finding the density matrix

In [9]:
rho = M / np.trace(M)
print('The density matrix is \n', rho)

The density matrix is 
 [[0.5        0.40793519]
 [0.40793519 0.5       ]]


Initializing a 5-qubit Quantum Circuit to calculate the eigen-values for the PCA covariance matrix

In [10]:
rho_eig_val, rho_eig_vec = np.linalg.eig(rho)

p_vec = np.concatenate((np.sqrt(rho_eig_val), np.sqrt(rho_eig_val)), axis=0)
U_vec = rho_eig_vec.reshape((4))

psi = p_vec * U_vec
print('The purified state is \n', psi)

rho_partial_trace = np.dot(psi.reshape((2,2)),psi.reshape((2,2)).transpose())
print('Verify the reduction to the original mixed state \n', rho_partial_trace)

The purified state is 
 [-0.21455164 -0.67377117  0.21455164 -0.67377117]
Verify the reduction to the original mixed state 
 [[0.5        0.40793519]
 [0.40793519 0.5       ]]


We will be measuring purity value through the Quantum Circuit

In [11]:
from qiskit import QuantumCircuit, execute, Aer, assemble

circ = QuantumCircuit(5, 1)

circ.initialize([1,0], (0,))
circ.initialize(psi, (1,2))
circ.initialize(psi, (3,4))
circ.h(0)
circ.cswap(0,1,3)
circ.h(0)
circ.measure(0,0)
circ.draw()

┌─────────────────┐                ┌───┐   ┌───┐┌─┐
q_0: ────────────────┤ Initialize(1,0) ├────────────────┤ H ├─■─┤ H ├┤M├
     ┌───────────────┴─────────────────┴───────────────┐└───┘ │ └───┘└╥┘
q_1: ┤0                                                ├──────X───────╫─
     │  Initialize(-0.21455,-0.67377,0.21455,-0.67377) │      │       ║ 
q_2: ┤1                                                ├──────┼───────╫─
     ├─────────────────────────────────────────────────┤      │       ║ 
q_3: ┤0                                                ├──────X───────╫─
     │  Initialize(-0.21455,-0.67377,0.21455,-0.67377) │              ║ 
q_4: ┤1                                                ├──────────────╫─
     └─────────────────────────────────────────────────┘              ║ 
c: 1/═════════════════════════════════════════════════════════════════╩═
                                                                      0

In [12]:
sim = Aer.get_backend('aer_simulator')

# circ.save_statevector()

qobj = assemble(circ)
counts = sim.run(circ, shots=10**7).result().get_counts()
print(counts)

{'1': 835994, '0': 9164006}


Calculating eigen-values from purity

In [13]:
purity = (counts['0'] - counts['1']) / (counts['0'] + counts['1'])
m_1 = (1 + np.sqrt(2 * purity - 1)) / 2 * np.trace(M)
m_2 = (1 - np.sqrt(2 * purity - 1)) / 2 * np.trace(M)
print('The eigenvalues obtained by the quantum PCA are \n', [m_1, m_2])

The eigenvalues obtained by the quantum PCA are 
 [1.9455477782619994, 0.19730936459514287]


Classical eigen-values

In [14]:
m, vec = np.linalg.eig(M)
idx = m.argsort()[::-1]
m = m[idx]
vec = vec[:,idx]
print('The eigenvalues obtained by classical diagonalization are \n', m)

The eigenvalues obtained by classical diagonalization are 
 [1.94557541 0.19728173]
